## Translation and Summary of Foreign News

The goal of this project will be to get a summary of news in foreign countries by automatically scraping a newspapers headlines or articles, and then translating the text with OpenAI and also providing a summary format.

### Data (Web Scraping)

Let's explore how we could set-up a scraping job (note this has its limits, not every website can be easily scraped, your results may vary on other newpaper sites).

In [1]:
import requests
import bs4

In [27]:
# This is just a limited example dictionary
# As each website is different, this gets harder with more countries!
country_newspapers = {"Spain":'https://elpais.com/', 
                       "France":"https://www.lemonde.fr/"
                     }

In [5]:
# Note how 
country = input("What country are you interested in for a news summary? ")

What country are you interested in for a news summary? France


In [6]:
url = country_newspapers[country]

In [7]:
url

'https://www.lemonde.fr/'

In [8]:
result = requests.get(url)

### Parsing HTML 

In [11]:
soup = bs4.BeautifulSoup(result.text,"lxml")

In [13]:
# France Le Monde
soup.select('.article__title-label')

[<p class="article__title-label">Dans la bataille de l’accès aux soins, le gouvernement face à la colère des médecins</p>,
 <span class="article__title-label">Pourquoi les prix de l’électricité s’envolent (et ne devraient pas redescendre)</span>,
 <span class="article__title-label">Les pompes à chaleur sont-elles le futur du chauffage ?</span>,
 <span class="article__title-label">« Moi, général de Gaulle » : l’appel du 18-Juin peut-il être reconstitué ?</span>,
 <span class="article__title-label">Les images des séismes qui ont tué des milliers de personnes en Syrie et en Turquie</span>]

Clearly, this won't be the same title tag for Spain's El Pais, let's quickly test that out, then we can recombine these tags into the dictionary to simplify and have all the information in one location.

In [29]:
spain_results = requests.get('https://elpais.com/')
soup = bs4.BeautifulSoup(spain_results.text,"lxml")

In [30]:
len(soup.select('.c_t'))

123

In [32]:
soup.select('.c_t')[:3]

[<h2 class="c_t"><a href="/internacional/2023-02-13/haiti-se-deshace.html">Haití se deshace</a></h2>,
 <h2 class="c_t c_t-sm"><a href="/internacional/2023-02-13/los-presos-politicos-buscan-una-casa-de-acogida-en-el-destierro-en-estados-unidos.html"><span class="c_t_i c_t_i-s _pr" name="elpais_ico"></span>Los presos políticos buscan una casa de acogida en el destierro en EE UU</a></h2>,
 <h2 class="c_t"><a href="/america-colombia/2023-02-13/el-gobierno-de-colombia-y-el-eln-buscan-en-mexico-un-cese-al-fuego-permanente-que-haga-despegar-el-proceso-de-paz.html">El Gobierno de Colombia y el ELN buscan en México un cese al fuego permanente que haga despegar el proceso de paz</a></h2>]

#### Grabbing Text

Now we just need to figure out how to grab these headlines text:

In [33]:
for tag in soup.select('.c_t')[:3]:
    print(tag.getText())

Haití se deshace
Los presos políticos buscan una casa de acogida en el destierro en EE UU
El Gobierno de Colombia y el ELN buscan en México un cese al fuego permanente que haga despegar el proceso de paz


### Combining Dictionary Tag Title and URL

In [34]:
# "Country" : (URL,HTML_TAG)
country_newspapers = {"Spain":('https://elpais.com/','.c_t'), 
                       "France":("https://www.lemonde.fr/",'.article__title-label')
                     }

## Translation via OpenAI

In [51]:
def create_prompt():
    # Get Country
    country = input("What country would you like a news summary for? ")
    # Get country's URL newspaper and the HTML Tag for titles
    try:
        url,tag = country_newspapers[country]
    except:
        print("Sorry that country is not supported!")
        return
    
    # Scrape the Website
    results = requests.get(url)
    soup = bs4.BeautifulSoup(results.text,"lxml")
    
    # Grab all the text
    country_headlines = ''
    for item in soup.select(tag)[:3]:
        country_headlines += item.getText()+'\n'
        
    prompt = "Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone:\n"
    return prompt + country_headlines

In [52]:
prompt = create_prompt()

What country would you like a news summary for? Spain


In [53]:
print(prompt)

Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone:
Haití se deshace
Los presos políticos buscan una casa de acogida en el destierro en EE UU
El Gobierno de Colombia y el ELN buscan en México un cese al fuego permanente que haga despegar el proceso de paz



In [37]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [54]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0.1, # Helps conversational tone a bit, optional
  max_tokens=200,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [56]:
print(response['choices'][0]['text'])


Language: Spanish

The situation in Haiti is deteriorating. Political prisoners are seeking refuge in the United States. The Colombian government and the ELN are looking to Mexico for a permanent ceasefire that will help the peace process move forward.
